# Test of the new driver for Tektronix 7k AWG

`Found TEKTRONIX AWG7122C Serial: B051135 FW: SCPI:99.0 FW:4.6.0.7 options: ['01', '06', '08', '09']`

## Basic commands and Hardware settings

### Basic commands

#### write()

In [1]:
myawg.write('*RST')

0

In [2]:
myawg.write('OUTPUT1:STATE ON')
myawg.get_errors()

True

In [3]:
myawg.load_waveform({1: '*Ramp10'})
myawg.write('OUTPUT1:STATE ON')

0

In [4]:
myawg.write('OUTPUT1:STATE OFF')

0

#### read()

In [5]:
myawg.write('*IDN?')
myawg.read()

'TEKTRONIX,AWG7122C,B051135,SCPI:99.0 FW:4.6.0.7 '

#### query()

In [7]:
myawg.query('*IDN?')

'TEKTRONIX,AWG7122C,B051135,SCPI:99.0 FW:4.6.0.7'

In [8]:
myawg.write('OUTPUT1:STATE ON')
myawg.query('OUTPUT1:STATE?')

'1'

In [9]:
myawg.write('OUTPUT1:STATE OFF')
myawg.query('OUTPUT1:STATE?')

'0'

#### reset()

In [17]:
myawg.reset()  # works

0

#### get_status()

In [14]:
myawg.pulser_off()
myawg.get_status()

(0, {-1: 'Failed Request or Communication', 0: 'Device has stopped, but can receive commands', 1: 'Device is active and running', 2: 'Device is waiting for trigger.'})

In [11]:
myawg.pulser_on()
myawg.get_status()

(2, {-1: 'Failed Request or Communication', 0: 'Device has stopped, but can receive commands', 1: 'Device is active and running', 2: 'Device is waiting for trigger.'})

#### get_errors()

In [15]:
myawg.get_errors()

False

In [22]:
myawg.get_errors()

True

```
AWG7122C error: -100 "Command error; Command not allowed - *IDN
AWG7122C error: -101 "Invalid character; Unexpected character found - OUTPUT1:STATE ?
```

### Hardware settings

#### get_constraints()

In [22]:
constr_obj = myawg.get_constraints()
type(constr_obj)

<class 'interface.pulser_interface.PulserConstraints'>

In [44]:
myawg.set_interleave(False)
constr_obj = myawg.get_constraints()

print('min: {} MHz'.format(constr_obj.sample_rate.min/1e6))
print('max: {} GHz'.format(constr_obj.sample_rate.max/1e9))
print('stp: {} MHz'.format(constr_obj.sample_rate.step/1e6))
print('def: {} GHz'.format(constr_obj.sample_rate.default/1e9))

min: 10.0 MHz
max: 12.0 GHz
stp: 10.0 MHz
def: 12.0 GHz


In [43]:
myawg.set_interleave(True)
constr_obj = myawg.get_constraints()

print('min: {} MHz'.format(constr_obj.sample_rate.min/1e6))
print('max: {} GHz'.format(constr_obj.sample_rate.max/1e9))
print('stp: {} MHz'.format(constr_obj.sample_rate.step/1e6))
print('def: {} GHz'.format(constr_obj.sample_rate.default/1e9))

min: 12000.0 MHz
max: 24.0 GHz
stp: 0.0005 MHz
def: 24.0 GHz


In [26]:
allowed_patterns = constr_obj.activation_config
for pattern_name, pattern_set in allowed_patterns.items():
    print('{}: {}'.format(pattern_name, pattern_set))

: set()
all: {'a_ch2', 'd_ch2', 'd_ch3', 'd_ch4', 'd_ch1', 'a_ch1'}
A1_M1_M2: {'d_ch1', 'd_ch2', 'a_ch1'}
A2_M3_M4: {'a_ch2', 'd_ch4', 'd_ch3'}
Two_Analog: {'a_ch2', 'a_ch1'}
Analog1: {'a_ch1'}
Analog2: {'a_ch2'}
A1_M1_M2_A2: {'a_ch2', 'd_ch1', 'd_ch2', 'a_ch1'}
A1_A2_M3_M4: {'a_ch2', 'd_ch4', 'a_ch1', 'd_ch3'}


#### set_mode(), get_mode()

In [31]:
myawg.set_mode('C')
myawg.get_mode()

'C'

In [27]:
myawg.set_mode('T')
myawg.get_mode()

'T'

In [29]:
myawg.set_mode('G')
myawg.get_mode()

'G'

In [30]:
myawg.set_mode('S')
myawg.get_mode()

'S'

#### set/get_sample_rate()

In [32]:
myawg.set_sample_rate(15e6)

15000000.0

In [33]:
myawg.set_sample_rate(-15e6)

10000000.0

In [34]:
myawg.set_interleave(True)
myawg.set_sample_rate(23e9)

23000000000.0

In [35]:
myawg.set_interleave(False)
myawg.set_sample_rate(23e9)

12000000000.0

#### set/get_analog_level()

In [36]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 0.6, 'a_ch2': 0.9},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=0.6, offset=0.0
a_ch2: amp=0.9, offset=0.0


In [37]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 1.5, 'a_ch2': 0.9},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=1.0, offset=0.0
a_ch2: amp=0.9, offset=0.0


In [38]:
ampl_res, offset_res = myawg.set_analog_level(
    amplitude={'a_ch1': 0.95, 'a_ch2': 0.85, 'AABBCC1': 0.5},
    offset={'a_ch1': 0, 'a_ch2': 0}  # Offset in not supported for options [02] and [06]
)

for ch_key in ampl_res:
    print('{}: amp={}, offset={}'.format(ch_key, ampl_res[ch_key], offset_res[ch_key]))

a_ch1: amp=0.95, offset=0.0
a_ch2: amp=0.85, offset=0.0


#### set/get_digital_level()

In [39]:
low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': -0.2, 'd_ch2': -0.1, 'd_ch3': 0.1, 'd_ch4': 0.2},
    high={'d_ch1': 0.8, 'd_ch2': 0.9, 'd_ch3': 1.1, 'd_ch4': 1.2}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=-0.2, high=0.8
d_ch2: low=-0.1, high=0.9
d_ch3: low=0.1, high=1.1
d_ch4: low=0.2, high=1.2


In [80]:
low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': 0.3, 'd_ch2': 0.1},
    high={'d_ch3': 0.6, 'd_ch4': 0.7}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=0.3, high=0.8
d_ch2: low=0.1, high=0.9
d_ch3: low=0.1, high=0.6
d_ch4: low=0.2, high=0.7


In [95]:
# NOTICE: if low and high values are too far apart (max difference is 1.4 V),
# the last-setted param is set to the specified value, but the first-setted one is pulled to
# meet the max distance restriction

low_res, high_res = myawg.set_digital_level(
    low={'d_ch1': -0.5},
    high={'d_ch1': 1.2}
)

for ch_key in low_res:
    print('{}: low={}, high={}'.format(ch_key, low_res[ch_key], high_res[ch_key]))

d_ch1: low=-0.2, high=1.2
d_ch2: low=0.1, high=0.9
d_ch3: low=0.1, high=0.6
d_ch4: low=0.2, high=0.7


#### set/get_active_channels()

##### Potential effects of not using copy.deepcopy() [since all elements are immutable, equivalent to copy.copy()]

In [3]:
myawg._ch_state_dict

{'a_ch1': False, 'a_ch2': False, 'd_ch1': False, 'd_ch2': False, 'd_ch3': False, 'd_ch4': False}

In [2]:
ch_dict = myawg.get_active_channels()

In [4]:
ch_dict['a_ch1'] = True

In [5]:
myawg._ch_state_dict

{'a_ch1': True, 'a_ch2': False, 'd_ch1': False, 'd_ch2': False, 'd_ch3': False, 'd_ch4': False}

##### Final version [with copy.deepcopy()]

In [1]:
myawg._get_all_channels()

['a_ch1', 'a_ch2', 'd_ch1', 'd_ch2', 'd_ch3', 'd_ch4']

In [2]:
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: False
a_ch2: False
d_ch1: False
d_ch2: False
d_ch3: False
d_ch4: False


In [3]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': False,
     'd_ch4': False    
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


In [4]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True,
    }
)

myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True    
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: True
d_ch4: True


`Transition to ({'d_ch3', 'd_ch1', 'a_ch1', 'd_ch4', 'd_ch2'}) active channels is not allowed according to constraints.
set_active_channels() aborted. Channel states not changed`

In [5]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': True,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': True,
     'd_ch4': True,
    }
)

myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': True,
     'd_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
     'd_ch5': False
    }
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: True
d_ch4: True


`Trying to (de)activate channels that are not present in AWG.
Setting of channel activation aborted.`

In [6]:
myawg.set_active_channels(
    {'a_ch1': False, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.set_active_channels(
    {'a_ch1': True}
)

ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: False
d_ch1: False
d_ch2: False
d_ch3: False
d_ch4: False


In [7]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.get_active_channels(['a_ch1', 'd_ch4'])

{'a_ch1': True, 'd_ch4': False}

In [8]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.get_active_channels(['a_ch1', 'd_ch4', 'd_ch5'])

{'a_ch1': True, 'd_ch4': False}

`Channel "d_ch5" is not available in AWG.
Ignore channel state request for this entry.`

#### pulser_on/off()

##### \_get_channel_state()

In [11]:
ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True
d_ch1: True
a_ch2: False
d_ch4: False
d_ch3: False


In [12]:
ch_st_dict = myawg._get_channel_state(['a_ch1', 'd_ch2', 'd_ch5'])
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True


`Channel "d_ch5" is not available in AWG.
Ignore channel state request for this entry.`

##### \_activate_channels()

In [14]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg._activate_channels()

ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True
d_ch1: True
a_ch2: True
d_ch4: False
d_ch3: False


In [18]:
# Analog Channel 1 has a wevoform loaded
# Manually deleted waveform from Analog Channel 2 (via AWG's screen)

myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg._activate_channels()

ch_st_dict = myawg._get_channel_state()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
d_ch2: True
d_ch1: True
a_ch2: False
d_ch4: False
d_ch3: False


```
AWG7122C error: 6000 "Other error; E11506 - OUTPUT2:STATE ON
-----------
Activation of channels by _activate_channels() failed
The expected state: {'a_ch1': True, 'a_ch2': True, 'd_ch1': True, 'd_ch2': True, 'd_ch3': False, 'd_ch4': False}
The obtained state: {'a_ch1': True, 'a_ch2': False, 'd_ch1': True, 'd_ch2': True, 'd_ch3': False, 'd_ch4': False}

```

##### pulser_on/off()

In [15]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.pulser_on()

2

In [16]:
myawg.pulser_off()
    
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: True
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


In [17]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': True,
     'd_ch2': True,
     'a_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.pulser_on()

2

In [18]:
myawg.pulser_off()
    
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

a_ch1: True
a_ch2: False
d_ch1: True
d_ch2: True
d_ch3: False
d_ch4: False


#### set/get_interleave()

In [9]:
myawg.set_interleave(state=True)

True

In [10]:
myawg.set_interleave(False)

False

#### set_lowpass_filter()   [not awailable on this AWG]

In [44]:
myawg.query('OUTPUT1:FILTER:LPASS:FREQUENCY?')

Traceback (most recent call last):
  File "C:\Users\Lukin SiV\qudi\logic\jupyterkernel\qzmqkernel.py", line 795, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-e158c0ec888f>", line 1, in <module>
    myawg.query('OUTPUT1:FILTER:LPASS:FREQUENCY?')
  File "C:\Users\Lukin SiV\qudi\hardware\awg\tektronix_awg7k.py", line 184, in query
    answer = self.awg.query(question)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 564, in query
    return self.read()
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 413, in read
    message = self._read_raw().decode(enco)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\resources\messagebased.py", line 386, in _read_raw
    chunk, status = self.visalib.read(self.session, size)
  File "C:\Anaconda\envs\qudi\lib\site-packages\pyvisa\ctwrapper\functions.py", line 1584, in read
    ret = library.viRead(session, buffer, count, byr

`OUTPut[n]:FILTer[:LPASs]:FREQuency`
This command and query sets or returns the low-pass filter frequency of the
filter. INFinity is same as Through (no filter). This command is not available on
instruments with option 02 or option 06

In [45]:
myawg.set_lowpass_filter(a_ch=1, cutoff_freq=100)

`Low-pass filter is not available for options [02] and [06]. set_lowpass_filter() request was ignored`

In [32]:
myawg.query('AWGC:RST?')

'0'

## Waveform and Sequence generation methods

### Waveform

#### write_waveform()

##### \_write_wfm()

In [1]:
import numpy as np

In [19]:
anlg_smpl_ar = np.zeros(10, dtype='float32')

for i in range(len(anlg_smpl_ar)):
    anlg_smpl_ar[i] = i/10
    
dg_smpl_ar = np.zeros(10, dtype='uint8')
dg_smpl_ar[0] = 0
dg_smpl_ar[1] = 1
dg_smpl_ar[2] = 2
dg_smpl_ar[3] = 3
dg_smpl_ar[4] = 0b00000000
dg_smpl_ar[5] = 0b00000001
dg_smpl_ar[6] = 0b00000010
dg_smpl_ar[7] = 0b00000011
dg_smpl_ar[8] = 0b00000010
dg_smpl_ar[9] = 0b00000001

In [18]:
0b00000010

2

In [21]:
dg_smpl_ar

array([0, 1, 2, 3, 0, 1, 2, 3, 2, 1], dtype=uint8)

In [20]:
anlg_smpl_ar

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype=float32)

In [27]:
myawg._write_wfm(
    filename='test1',
    analog_samples=anlg_smpl_ar,
    marker_bytes=dg_smpl_ar,
    is_first_chunk=True,
    is_last_chunk=True,
    total_number_of_samples=10
)

In [5]:
myawg.get_sample_rate()

12000000000.0

Method works correctly

![title](2.png)

##### write_waveform()

In [115]:
a_samp = {
    'a_ch1': np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype='float32'),
    'a_ch2': np.array([-0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9], dtype='float32')
}

d_samp = {
    'd_ch1': np.array([True, False, True, False, True, False, True, False, True, False]),
    'd_ch2': np.array([False, True, False, True, False, True, False, True, False, True]),
    'd_ch3': np.array([True, True, False, False, True, True, False, False, True, True]),
    'd_ch4': np.array([False, False, True, True, False, False, True, True, False, False])
}

In [116]:
myawg.write_waveform(
    name='test2',
    analog_samples=a_samp,
    digital_samples=d_samp,
    is_first_chunk=True,
    is_last_chunk=True,
    total_number_of_samples=10
)

(10, ['test2_ch1', 'test2_ch2'])

Method works correctly

![title](4.png)

###### Warning if active channel set doesn't match the waveform

In [6]:
a_samp = {
    'a_ch1': np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype='float32'),
    'a_ch2': np.array([-0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9], dtype='float32')
}

d_samp = {
    'd_ch1': np.array([True, False, True, False, True, False, True, False, True, False]),
    'd_ch2': np.array([False, True, False, True, False, True, False, True, False, True]),
    'd_ch3': np.array([True, True, False, False, True, True, False, False, True, True]),
    'd_ch4': np.array([False, False, True, True, False, False, True, True, False, False])
}

In [9]:
ch_st_dict = myawg.get_active_channels()
for ch_name, ch_state  in ch_st_dict.items():
    print('{}: {}'.format(ch_name, ch_state))

myawg.write_waveform(
    name='test2',
    analog_samples=a_samp,
    digital_samples=d_samp,
    is_first_chunk=True,
    is_last_chunk=True,
    total_number_of_samples=10
)

a_ch1: True
a_ch2: True
d_ch1: False
d_ch2: False
d_ch3: False
d_ch4: False


(10, ['test2_ch1', 'test2_ch2'])

![title](6.png)

###### Error if not all arrays have same length

In [3]:
a_samp = {
    'a_ch1': np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype='float32'),
    'a_ch2': np.array([-0.0, -0.1, -0.2, -0.3, -0.4, -0.5], dtype='float32')
}

d_samp = {
    'd_ch1': np.array([True, False, True, False, True, False, True, False, True, False]),
    'd_ch2': np.array([False, True, False, True, False, True, False, True, False, True]),
    'd_ch3': np.array([True, True, False, False, True, True, False, False, True, True]),
    'd_ch4': np.array([False, False, True, True, False, False, True, True, False, False])
}

In [194]:
myawg.write_waveform(
    name='test2',
    analog_samples=a_samp,
    digital_samples=d_samp,
    is_first_chunk=True,
    is_last_chunk=True,
    total_number_of_samples=10
)

(10, ['test2_ch1', 'test2_ch2'])

![title](8.png)

In [5]:
a_samp = {
    'a_ch1': np.array([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype='float32'),
    'a_ch2': np.array([-0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9], dtype='float32')
}

d_samp = {
    'd_ch1': np.array([True, False, True, False, True, False, True]),
    'd_ch2': np.array([False, True, False, True, False, True, False, True, False, True]),
    'd_ch3': np.array([True, True, False, False, True, True, False, False, True, True]),
    'd_ch4': np.array([False, False, True, True, False, False, True, True, False, False])
}

In [6]:
myawg.write_waveform(
    name='test2',
    analog_samples=a_samp,
    digital_samples=d_samp,
    is_first_chunk=True,
    is_last_chunk=True,
    total_number_of_samples=10
)

(-1, [])

![title](8.png)

#### load_waveform()

In [7]:
myawg.load_waveform(
    {1: 'test2_ch1', 2: 'test2_ch2'}
)

({1: 'test2_ch1', 2: 'test2_ch2'}, 'waveform')

Method works correctly

![title](5.png)

##### Error if the channel is not active

In [11]:
myawg.set_active_channels(
    {'a_ch1': True, 
     'a_ch2': False,
     'd_ch1': False,
     'd_ch2': False,
     'd_ch3': False,
     'd_ch4': False    
    }
)
    
myawg.load_waveform(
    {1: 'test2_ch1', 2: 'test2_ch2'}
)

({1: '', 2: ''}, 'waveform')

![title](10.png)

#### get_waveform_names()

In [14]:
myawg.get_waveform_names()

['*Clock960', '*DC', '*PRBS - X^15+X^14+1', '*PRBS - X^7+X^6+1', '*PRBS - X^9+X^5+1', '*PRBS - X^9+X^8+X^5+X^4+1', '*Ramp10', '*Ramp100', '*Ramp1000', '*Ramp960', '*Sine10', '*Sine100', '*Sine1000', '*Sine36', '*Sine360', '*Sine3600', '*Sine960', '*Square10', '*Square100', '*Square1000', '*Square960', '*Triangle10', '*Triangle100', '*Triangle1000', '*Triangle960', 'laser_on_ch1', 'laser_on_ch2', 'test2_ch1', 'test2_ch2']

In [15]:
for wfm_name in myawg.get_waveform_names():
    print(wfm_name)

*Clock960
*DC
*PRBS - X^15+X^14+1
*PRBS - X^7+X^6+1
*PRBS - X^9+X^5+1
*PRBS - X^9+X^8+X^5+X^4+1
*Ramp10
*Ramp100
*Ramp1000
*Ramp960
*Sine10
*Sine100
*Sine1000
*Sine36
*Sine360
*Sine3600
*Sine960
*Square10
*Square100
*Square1000
*Square960
*Triangle10
*Triangle100
*Triangle1000
*Triangle960
laser_on_ch1
laser_on_ch2
test2_ch1
test2_ch2


#### delete_waveform()

In [16]:
myawg.delete_waveform('laser_on_ch1')

['laser_on_ch1']

In [17]:
for wfm_name in myawg.get_waveform_names():
    print(wfm_name)

*Clock960
*DC
*PRBS - X^15+X^14+1
*PRBS - X^7+X^6+1
*PRBS - X^9+X^5+1
*PRBS - X^9+X^8+X^5+X^4+1
*Ramp10
*Ramp100
*Ramp1000
*Ramp960
*Sine10
*Sine100
*Sine1000
*Sine36
*Sine360
*Sine3600
*Sine960
*Square10
*Square100
*Square1000
*Square960
*Triangle10
*Triangle100
*Triangle1000
*Triangle960
laser_on_ch2
test2_ch1
test2_ch2


### Wfm technical

In [89]:
myawg._get_filenames_on_device()

['AA_only_a_ch1_ch1.wfm', 'AA_only_a_ch1_test2a_ch1.wfm', 'AA_only_a_ch1_test2b_ch1.wfm', 'AA_only_a_ch1_test2_ch1.wfm', 'AA_only_a_ch1_testA_ch1.wfm', 'AA_only_a_ch1_test_ch1.wfm', 'AA_only_ch1.wfm', 'AA_only_ch2.wfm', 'AA_only__ch2.wfm', 'AA_test1_wfm_ch1.wfm', 'AA_test1_wfm_ch2.wfm', 'GUI_test_pulse_block_ch1.wfm', 'GUI_test_pulse_block_ch2.wfm', 'laser_on_ch1.wfm', 'laser_on_ch2.wfm', 'rabi_ch1.wfm', 'rabi_ch2.wfm', 'test100_ch1.wfm', 'test101_ch1.wfm', 'test102_ch1.wfm', 'test103_ch1.wfm', 'test104_ch1.wfm', 'test105_ch1.wfm', 'test106_ch1.wfm', 'test107_ch1.wfm', 'test108_ch1.wfm', 'test109_ch1.wfm', 'test10_ch1.wfm', 'test110_ch1.wfm', 'test111_ch1.wfm', 'test112_ch1.wfm', 'test113_ch1.wfm', 'test114_ch1.wfm', 'test115_ch1.wfm', 'test116_ch1.wfm', 'test117_ch1.wfm', 'test118_ch1.wfm', 'test119_ch1.wfm', 'test11_ch1.wfm', 'test120_ch1.wfm', 'test121_ch1.wfm', 'test122_ch1.wfm', 'test123_ch1.wfm', 'test124_ch1.wfm', 'test125_ch1.wfm', 'test126_ch1.wfm', 'test127_ch1.wfm', 'test128

In [91]:
myawg._delete_file('test199_ch1.wfm')  # Works correctly

### Sequence

#### write_sequence()

In [174]:
seq_param_list = [
    (('test2_ch1', 'test2_ch2'), {'repetitions': 2, 'wait_for': 'ON', 'go_to': 3, 'event_jump_to': 4})
]

myawg.write_sequence(name='sequenceA', sequence_parameter_list=seq_param_list)

1

![title](31.png)

In [175]:
seq_param_list = [
    (('test2_ch1', 'test2_ch2'), {'repetitions': -1, 'wait_for': 'ON', 'go_to': 3, 'event_jump_to': 4})
]

myawg.write_sequence(name='sequenceA', sequence_parameter_list=seq_param_list)

1

![title](29.png)

The set of channels if determined by "wfm_name_ch#", so it's important to keep this naming format

![title](27.png)

In [195]:
seq_param_list = [
    (('test2_ch1', 'test2_ch2'), {'repetitions': -1, 'wait_for': 'OFF', 'go_to': 1, 'event_jump_to': 0})
]

myawg.write_sequence(name='sequenceA', sequence_parameter_list=seq_param_list)

1

![title](30.png)

#### load_sequence()

In [196]:
myawg.get_sequence_names()

['sequenceA']

In [199]:
myawg.load_sequence('sequenceA')

({1: 'sequenceA', 2: 'sequenceA'}, 'sequence')

In [200]:
myawg.get_loaded_assets()

({1: 'sequenceA', 2: 'sequenceA'}, 'sequence')

#### set/get_sequence_jump_mode()

Even if the run mode is different from Sequence,
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [3]:
myawg.set_sequence_jump_mode('EJUM')
myawg.get_sequence_jump_mode()

'EJUM'

In [4]:
myawg.set_sequence_jump_mode('DJUM')
myawg.get_sequence_jump_mode()

'DJUM'

#### sequence_set_waveform()

Even if the run mode is different from Sequence, 
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [48]:
myawg.get_waveform_names()

['*Clock960', '*DC', '*PRBS - X^15+X^14+1', '*PRBS - X^7+X^6+1', '*PRBS - X^9+X^5+1', '*PRBS - X^9+X^8+X^5+X^4+1', '*Ramp10', '*Ramp100', '*Ramp1000', '*Ramp960', '*Sine10', '*Sine100', '*Sine1000', '*Sine36', '*Sine360', '*Sine3600', '*Sine960', '*Square10', '*Square100', '*Square1000', '*Square960', '*Triangle10', '*Triangle100', '*Triangle1000', '*Triangle960', 'laser_on_ch2', 'test2_ch1', 'test2_ch2']

In [60]:
myawg.write('SEQUENCE:LENGTH 1')
myawg.sequence_set_waveform(wfm_name='test2_ch1', elem_num=1, chnl_num=1)
myawg.sequence_set_waveform(wfm_name='test2_ch2', elem_num=1, chnl_num=2)

0

![title](14.png)

#### sequence_set_repetitions() 

Even if the run mode is different from Sequence, 
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [12]:
myawg.sequence_set_repetitions(1, -1)

0

![title](15.png)

In [14]:
myawg.sequence_set_repetitions(1, 2)

0

![title](16.png)

#### sequence_set_goto()

Even if the run mode is different from Sequence,
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [18]:
myawg.write('SEQUENCE:LENGTH 2')
myawg.sequence_set_waveform(wfm_name='test2_ch1', elem_num=1, chnl_num=1)
myawg.sequence_set_waveform(wfm_name='test2_ch2', elem_num=1, chnl_num=2)
myawg.sequence_set_waveform(wfm_name='test2_ch1', elem_num=2, chnl_num=1)
myawg.sequence_set_waveform(wfm_name='test2_ch2', elem_num=2, chnl_num=2)

0

In [23]:
myawg.sequence_set_goto(elem_num=2, goto=1)

0

![title](17.png)

In [24]:
myawg.sequence_set_goto(elem_num=2, goto=-1)

0

![title](18.png)

#### sequence_set_event_jump()

Even if the run mode is different from Sequence, and/or jump mode is different from Event Jump, 
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [36]:
myawg.sequence_set_event_jump(elem_num=1, jump_to=0)

0

![title](19.png)

In [37]:
myawg.sequence_set_event_jump(elem_num=1, jump_to=2)

0

![title](20.png)

In [33]:
myawg.sequence_set_event_jump(elem_num=1, jump_to=-1)

0

![title](21.png)

#### sequence_set_wait_trigger()

Even if the run mode is different from Sequence, and/or jump mode is different from Event Jump, 
the method works correctly: the applied value is applied, and is on the place when one swithces to the right mode

In [46]:
myawg.sequence_set_wait_trigger(elem_num=1, trigger='ON')

0

![title](22.png)

In [48]:
myawg.sequence_set_wait_trigger(elem_num=1, trigger='OFF')

0

![title](23.png)

In [47]:
myawg.sequence_set_wait_trigger(elem_num=1, trigger='A')

-1

![title](24.png)

In [5]:
myawg.set_mode('S')

#### set_jump_timing()

In [107]:
myawg.set_jump_timing(synchronous=True)
myawg.query('EVEN:JTIM?')

'SYNC'

In [108]:
myawg.set_jump_timing(synchronous=False)
myawg.query('EVEN:JTIM?')

'ASYN'

#### make_sequence_continuous()

In [70]:
myawg.make_sequence_continuous()

2

![title](25.png)

#### delete_sequence()

In [59]:
myawg.delete_sequence()

[]

Method works correctly

#### get_sequencer_mode()

In [71]:
myawg.get_sequencer_mode()

'Software-Sequencer'

#### force_jump_sequence()

In [84]:
myawg.force_jump_sequence(target=2, channel=1)

Something doesn't work

![title](26.png)

### Wfm and Seq common Technical

#### get_loaded_assets()

In [98]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': False,
     'd_ch2': False,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.set_mode('C')
myawg.get_loaded_assets()

({1: 'test2_ch1', 2: 'test2_ch2'}, 'waveform')

In [100]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': False,
     'd_ch2': False,
     'a_ch2': False,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.set_mode('C')
myawg.get_loaded_assets()

({1: 'test2_ch1'}, 'waveform')

In [99]:
myawg.set_active_channels(
    {'a_ch1': True,
     'd_ch1': False,
     'd_ch2': False,
     'a_ch2': True,
     'd_ch3': False,
     'd_ch4': False,
    }
)

myawg.set_mode('S')
myawg.get_loaded_assets()

({}, 'sequence')

#### clear_all()

In [101]:
myawg.get_waveform_names()

['*Clock960', '*DC', '*PRBS - X^15+X^14+1', '*PRBS - X^7+X^6+1', '*PRBS - X^9+X^5+1', '*PRBS - X^9+X^8+X^5+X^4+1', '*Ramp10', '*Ramp100', '*Ramp1000', '*Ramp960', '*Sine10', '*Sine100', '*Sine1000', '*Sine36', '*Sine360', '*Sine3600', '*Sine960', '*Square10', '*Square100', '*Square1000', '*Square960', '*Triangle10', '*Triangle100', '*Triangle1000', '*Triangle960', 'laser_on_ch2', 'test2_ch1', 'test2_ch2']

In [104]:
myawg.load_waveform({1: '*Sine360'})

({1: '*Sine360'}, 'waveform')

In [110]:
myawg.clear_all()

0

In [87]:
myawg.write('SLIS:SUBS:DEL ALL')
myawg.get_errors()

False